In [1]:
import soundfile as sf
import os
import numpy as np
import librosa
import torch
from config import fft_num, win_shift
base_path = r"D:\\work\\speechEnhancement\\datasets\\voicebank_demand"
audio_name = "p232_001.wav"
noisy_audio_path = os.path.join(base_path, "noisy_testset_wav", audio_name)

feat_wav, _ = sf.read(noisy_audio_path)  # (27861,)
c = np.sqrt(len(feat_wav) / np.sum((feat_wav ** 2.0))) 
feat_wav = feat_wav * c         # (27861,)
feat_x = librosa.stft(feat_wav, n_fft=fft_num, hop_length=win_shift, window='hann').T  # (175, 161)  (seq_len, freq)

#compress
feat_x, phase_x = np.abs(feat_x)**0.5, np.angle(feat_x)

feat_x = torch.FloatTensor(feat_x)
phase_x = torch.FloatTensor(phase_x)
feat_x = torch.stack((feat_x * torch.cos(phase_x), feat_x * torch.sin(phase_x)), dim=0) # (2, 175, 161) 

In [11]:
import torchaudio
from config import win_size
feat_wav, _ = torchaudio.load(noisy_audio_path)
feat_wav = feat_wav.squeeze(0)
c = torch.sqrt(feat_wav.shape[0] / torch.sum((feat_wav ** 2.0)))
feat_wav = feat_wav * c

feat_x = torch.stft(feat_wav, n_fft=fft_num, hop_length=win_shift, win_length=win_size, window=torch.hann_window(win_size), return_complex=True).T
feat_x, phase_x = torch.sqrt(torch.abs(feat_x)), torch.angle(feat_x)
feat_x = torch.stack((feat_x * torch.cos(phase_x), feat_x * torch.sin(phase_x)), dim=0)
print(feat_x.shape)

torch.Size([2, 175, 161])


In [3]:
from einops import rearrange
a = torch.rand([2, 4, 3])
print(a)
a.tolist()

tensor([[[0.5822, 0.8967, 0.9372],
         [0.5025, 0.4827, 0.5132],
         [0.7758, 0.4996, 0.1727],
         [0.9817, 0.6477, 0.0213]],

        [[0.4846, 0.4733, 0.2612],
         [0.8286, 0.2368, 0.6475],
         [0.4967, 0.0924, 0.8803],
         [0.0429, 0.5348, 0.5149]]])


[[[0.5822160840034485, 0.8966960310935974, 0.9372186064720154],
  [0.5025443434715271, 0.482688307762146, 0.5132083296775818],
  [0.7757612466812134, 0.49961501359939575, 0.17272770404815674],
  [0.9816822409629822, 0.6477407813072205, 0.021299123764038086]],
 [[0.4846345782279968, 0.4733104705810547, 0.2611798644065857],
  [0.8286473155021667, 0.23679548501968384, 0.6475166082382202],
  [0.4967333674430847, 0.09240090847015381, 0.8803408741950989],
  [0.04286831617355347, 0.5347602963447571, 0.5148664116859436]]]

In [5]:
seq_len = [2,3,4]
a = [torch.rand([2, q, 3]) for q in seq_len]
print(a)

[tensor([[[0.6514, 0.8858, 0.0766],
         [0.6374, 0.9574, 0.2171]],

        [[0.2564, 0.4451, 0.4436],
         [0.4918, 0.9460, 0.5537]]]), tensor([[[0.4676, 0.9347, 0.1520],
         [0.7374, 0.6358, 0.5229],
         [0.7143, 0.8532, 0.0647]],

        [[0.6113, 0.9465, 0.5271],
         [0.4983, 0.9589, 0.7804],
         [0.0495, 0.6888, 0.7786]]]), tensor([[[0.6230, 0.6114, 0.9438],
         [0.1533, 0.8112, 0.3777],
         [0.7447, 0.7943, 0.1035],
         [0.7700, 0.0407, 0.2573]],

        [[0.8783, 0.6899, 0.7823],
         [0.4480, 0.5782, 0.8545],
         [0.3409, 0.1223, 0.5808],
         [0.4136, 0.1618, 0.3451]]])]


In [7]:
b1 = []
for a_ in a:
    b1.append(torch.cat([a_[0], a_[1]], dim=1))


In [10]:
b = [rearrange(a_, "N L C -> L (N C)") for a_ in a]
from torch.nn.utils.rnn import pad_sequence
c = pad_sequence(b, batch_first=True)
print(c.shape)
d = rearrange(c, "B L (N C) -> B N L C", N=2)
print(d.shape)
print(d)

torch.Size([3, 4, 6])
torch.Size([3, 2, 4, 3])
tensor([[[[0.6514, 0.8858, 0.0766],
          [0.6374, 0.9574, 0.2171],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.2564, 0.4451, 0.4436],
          [0.4918, 0.9460, 0.5537],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]],


        [[[0.4676, 0.9347, 0.1520],
          [0.7374, 0.6358, 0.5229],
          [0.7143, 0.8532, 0.0647],
          [0.0000, 0.0000, 0.0000]],

         [[0.6113, 0.9465, 0.5271],
          [0.4983, 0.9589, 0.7804],
          [0.0495, 0.6888, 0.7786],
          [0.0000, 0.0000, 0.0000]]],


        [[[0.6230, 0.6114, 0.9438],
          [0.1533, 0.8112, 0.3777],
          [0.7447, 0.7943, 0.1035],
          [0.7700, 0.0407, 0.2573]],

         [[0.8783, 0.6899, 0.7823],
          [0.4480, 0.5782, 0.8545],
          [0.3409, 0.1223, 0.5808],
          [0.4136, 0.1618, 0.3451]]]])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous